# Functions

Welcome to the first lecture on the functional programming section. This lecture introduces functions, clears the confusion around functions and methods (defs) and also introduces more advanced concepts around functions in Scala.

### Anatomy of a Function

In a functional programming language people like to work or get more out of functions. For example we might want to pass a function as a parameter or simply use a function like any other value. With this in mind, we'd like functions to be ordinary first-class elements in our programming language.

Cool, cool but how to we do that? Scala works on top of the JVM which was designed obviously for Java. The first class elements in Java are objects (where an object is an instance of some class). So how do we level-up functions?

Traditionally you would have perhaps a singleton object that would represent a function and you would pass around that function wherever it's required. In Java, a really simplified version could look like this: 

```java
interface Function<A, B> {
    public B apply(A elem);
}

class Double implements Function<Integer, Integer> {
    public static final Double INSTANCE = new Double();
    private Double() {}
    public Integer apply(Integer elem) {
        return elem * 2;
    }
}
```

The problem with this (in Java at least) is that it doesn't really "feel" like a function when you call it:

```java
public class Main {
    public static void main(String[] args) {
        Double double = Double.INSTANCE;
        double.apply(2); // Not that natural, we'd like something like this: double(2)
    }
}
```

Fortunately Scala has a nice way of solving this problem. Re-thinking our implementation in Scala, we would end up with:

```scala
object Double {
  def apply(elem: Int): Int = elem * 2
}
```

That's all you need to do because we're using singleton objects. And using it is quite simple as well:

```scala
object Main extends App {
  Double(2) // Feels more natural
}
```

How is this possible? Remember apply? `apply` is the method that makes objects callable. So when an object has an apply method, the Scala compiler will automatically replace any calls of the form `instace(param)` with `instance.apply(param)`.

This brings us to the next point - functions in Scala are instances of objects which define an apply method. Their base trait actually looks like this:

```scala
trait Function1[-A, +B] {
  def apply(elem: A): B
}
```

Just ignore the variance (-, +) on types for now. We'll have an entire section dedicated to it later. There are predefined traits for functions with up to 22 parameters. That one looks quite ugly:

```scala
trait Function22[-T1, -T2, ..., -T22, +R] {
  def apply(x1: T1, x2: T2, ..., x22: T22): R
}
```

So bottom line here is that in Scala all functions are instances of `Function1`, ..., `Function22` traits. The compiler heavy-lifts what it founds as "callable code" into actual method calls of the apply method defined on your instances. Methods are just `def`s - every `def` defines a method. That's it.

Before we move on to our first exercises, let us create a succ function that gives us the next consecutive integer of our argument:

In [ ]:
val succ: Function1[Int, Int] = new Function1[Int, Int] {
  override def apply(elem: Int): Int = elem + 1
}
succ(2)

Scala has a nice syntactic sugar for function types. The following types are equivalent:

```scala
Function1[Int, Int]
Int => Int

Function2[Int, Int, Int]
(Int, Int) => Int
```

And so on, you get the point here. You will use this syntax quite often. For example, we are building a client library that performs HTTP calls. If you think about a `RequestHandler`, it's actually just a function that receives an `HttpRequest` and returns an `HttpResponse`:

```scala
class HttpRequest
class HttpResponse

trait RequestHandler extends (HttpRequest => HttpResponse)
```

Throughout this lecture, we'll find other places where this is useful as well.

### Exercises

Ok, now for our first exercises we just want to get used to this syntax. Define the following functions as instances of `FunctionN`, where `N` is the number of arguments of the function. Use the syntactic sugar to specify the type of the function:

1. Define the function `isEven` that takes an Integer and returns true if it's an even number.
2. Define the function `add` that takes two Integers and returns their sum.
3. Re-define the factorial method as a function. Use tail-recursivity for the actual implementation.

In [ ]:
// Solve exercises below

### Anonymous Functions

In practice we don't use this syntax too often. Instead, Scala allows you to define anonymous functions. They look like lambdas from any programming language in general - in fact, you'll often hear about anonymous lambdas which refer to the same thing.

```scala
// They can span on multiple lines if you need them to.
val succ: Int => Int = { x =>
  x + 1
}

// But one-liners are ok as well for simple things.
val succ: Int => Int = { x => x + 1 }
```

Don't forget to put the type specification, otherwise the compiler will be confused:

In [ ]:
val succ = { x => x + 1 }

But if the type specification is not good for you, you can always give the compiler a hint and it will figure out the rest:

```scala
val succ = { x: Int => x + 1 }
```

Oh, and if you have multiple arguments, put them between round brackets `()`:

```scala
val mod = { (x: Int, y: Int) => x % y }
```

Practice again with the same syntax. We really want to get use to it so take all the exercises above and redefine the functions to be anonymous functions (lambdas).

### Higher-Order Functions

Ok, right now you're probably wondering: "How does this help me? Why does it make sense to create instances of functions?" No worries, your answer is what follows.

With functions being instances of objects we could pass them on to other functions, as arguments. Or we could even return them as values. That's why we say that functions are first-class citizens in Scala - they are just like any other object instance. You can return them as results or pass them as arguments.

All functions that receive another function as a parameter or that return functions as their results are said to be higher-order functions. You may not see it yet, but this is a very powerful tool.

Consider the `map` function, available on List:

```scala
trait List[+A] {
  def map[B](f: A => B): List[B]
}
```

Notice how it receives another function `f` as a parameter. This makes `map` a higher-order function. Now why are higher-order functions so cool? Well, they enable new layers of abstraction. Let's try it:

In [ ]:
val numbers = List(1, 2, 3)
numbers.map(x => x + 1)

Ok, what the hell happened there? The map function is pretty cool. It uses the argument function `f` to map all elements `x` in the list to their `f(x)` values. Take a moment and notice how we passed an anonymous function as the argument of map.

And there's an even shorter way to define an anonymous function - you can just do:

In [ ]:
val numbers = List(1, 2, 3)
numbers.map(_ + 1)

That's the shortest you can get in Scala. Be careful though, as everything is inferred by the compiler and is contextual. The compiler can "guess" the simple things you need, but not things such as:

In [ ]:
// Nope, not gonna' work.
val add = _ + _

In [ ]:
// Again, give it a small hint.
val add: (Int, Int) => Int = _ + _
add(2, 3)

Higher-order functions are found in all collections. They are very useful to provide abstractions over arbitrary types. We'll learn more higher-order functions in the Collections notebook.

### Curried Functions

Curried functions are often thought of as functions with multiple parameter lists. Yes, Scala allows defining multiple parameter lists. For example, the function add can be re-written as:

In [ ]:
def add(x: Int)(y: Int): Int = x + y

The type of this function is `Int => Int => Int` or a function that receives an integer and returns another function which receives an integer and returns an integer. Inception, right?

Curried functions enable further specializations. Using our add function defined above, we can re-define our function succ simply as:

In [ ]:
def succ(x: Int): Int = add(1)(x)

That's it! Before we can define our own higher-order / curried functions it would be a good idea to experiment with a few existing ones. Check out the next notebooks which focus on specific higher-order functions or collections and try them out!